In [1]:
# GET  api/v1/townPlanning/commuteSchool/areaPopulationCircle

In [1]:
import requests
import numpy as np
import pandas as pd
import json
import folium
from resaspy import Resaspy
from functools import reduce

# Let's retrieve date and night population per ward of Tokyo from RESAS API

In [2]:
with open("apikey.txt", "r") as f:
    api_key = f.read()

In [3]:
resas_key = api_key
resas = Resaspy(resas_key)
cities = resas.cities(13)["result"] # pick up tokyo (=13)
city_codes = []
for city in cities:
    city_codes.append(city["cityCode"])

In [4]:
url = "https://opendata.resas-portal.go.jp"
#end = "api/v1/municipality/residentTaxCorporate/perYear"
#end = "api/v1/municipality/propertyTax/perYear"
end = "api/v1/townPlanning/commuteSchool/areaPopulationCircle"
#params = {"prefecture_cd":13, "city_cd":city_codes, "mode":2, "year":2015} # https://opendata.resas-portal.go.jp/docs/api/v1/townPlanning/commuteSchool/areaPopulationCircle.html
headers = {"X-API-KEY": api_key}

In [5]:
#url_ = url + "/" + end + "?" + "{}={}".format(param, 13103)
#requests.get(url_, headers=headers).json()["result"]

In [5]:
tests = {"prefecture_cd":13, "city_cd":13101, "mode":2, "year":2015} 
"?".join([k+"="+str(v) for k,v in zip(tests.keys(), tests.values())])

'prefecture_cd=13?city_cd=13101?mode=2?year=2015'

In [6]:
ress = []
for city in city_codes:
    params = {"prefecture_cd":13, "city_cd":int(city), "mode":2, "year":2015}
    params = "&".join([k+"="+str(v) for k,v in zip(params.keys(), params.values())])
    url_ = url + "/" + end + "?" + params
    res = requests.get(url_, headers=headers).json()["result"]
    try:
        df = pd.DataFrame(res).loc[0, ["year", "city", "noonDataSum", "nightDataSum", "dayNightRate"]]
        ress.append(df)
    except:
        pass

In [7]:
df = reduce((lambda x, y: pd.concat([x,y], axis=1)), ress).T

In [8]:
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)

In [9]:
df.head()

,year,city,noonDataSum,nightDataSum,dayNightRate
0,2015,13101,853068,58406,1460.58
1,2015,13102,608603,141183,431.07
2,2015,13103,940785,243283,386.7
3,2015,13104,775549,333560,232.51
4,2015,13105,346132,219724,157.53


# Choropleth with tokyo23.json

In [10]:
# read geojson file
geojson = "../geojson/JapanCityGeoJson/geojson/13/tokyo23.json"
tokyo23_location = [35.658593, 139.745441]

m = folium.Map(location=tokyo23_location,tiles='cartodbpositron',zoom_start=10) 

folium.Choropleth(
    geo_data=geojson,
    name='choropleth',
    data=df,
    columns=['city', 'noonDataSum'],  # 行政区分コードと表示データ
    key_on='feature.id',
    fill_color='YlOrRd',
    threshold_scale=[0,125*1000, 250*1000, 500*1000, 750*1000, 1000*1000], # 境界値を指定
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='日中人口'    # 凡例表示名
).add_to(m)

folium.LayerControl().add_to(m)
m

#### Some errors?
#### Here is a tip: https://teratail.com/questions/195732

# Correct tokyo23.json

In [258]:
with open(geojson) as f:
    d = json.load(f)

features = d['features']
# idリスト
ids = sorted(set([d['id'] for d in features]))
new_features = []
for id in ids:
    data = [d['geometry']['coordinates'] for d in features if d['id']==id]
    if len(data) == 1:
        feature = dict(
            type = "Feature",
            geometry = dict(
                type = "Polygon",
                coordinates = data[0],
            ),
            id = id,
        )
    else:
        feature = dict(
            type = "Feature",
            geometry = dict(
                type = "MultiPolygon",
                coordinates = data,
            ),
            id = id,
        )
    new_features.append(feature)
d['features'] = new_features

with open('../geojson/JapanCityGeoJson/geojson/13/new_tokyo23.json', 'w') as f:
    json.dump(d, f)

# Choropleth with corrected tokyo23.json (new_tokyo23.json)

In [11]:
# read geojson file
geojson = "../geojson/JapanCityGeoJson/geojson/13/new_tokyo23.json"
tokyo23_location = [35.658593, 139.745441]

m = folium.Map(location=tokyo23_location,tiles='cartodbpositron',zoom_start=10) 

folium.Choropleth(
    geo_data=geojson,
    name='日中人口',
    data=df,
    columns=['city', 'noonDataSum'],  # 行政区分コードと表示データ
    key_on='feature.id',
    fill_color='YlOrRd',
    threshold_scale=[0,125*1000, 250*1000, 500*1000, 750*1000, 1000*1000], # 境界値を指定
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='人口'    # 凡例表示名
).add_to(m)

folium.Choropleth(
    geo_data=geojson,
    name='夜間人口',
    data=df,
    columns=['city', 'nightDataSum'],  # 行政区分コードと表示データ
    key_on='feature.id',
    fill_color='YlOrRd',
    threshold_scale=[0,125*1000, 250*1000, 500*1000, 750*1000, 1000*1000], # 境界値を指定
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='人口'    # 凡例表示名
).add_to(m)

folium.LayerControl().add_to(m)
m

# Done!
#### I've sent an issue regarding this to the original repository: https://github.com/niiyz/JapanCityGeoJson/issues/18